Imports

In [ ]:
!pip install ffmpeg-python
!pip install av
!pip install -q -U google-generativeai
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate

In [ ]:
import os
import cv2
import time
import torch
import random
import ffmpeg
import warnings
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
import soundfile as sf
from json import loads,dumps
import matplotlib.pyplot as plt
from scipy.signal import resample
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold

In [ ]:
df = pd.read_csv('/kaggle/input/youtube-data/ground_labels_new.csv') 

df['Primary Label'] = df['Primary Label'].str.lower() 
df = df[df['Primary Label'].isin(['appropriate', 'inappropriate'])]

ids_df = pd.read_csv('/kaggle/input/dynamic-few-shot/gemini-uniform-remaining-3.csv') 
ids_df['Video Id'] = ids_df['Video Id'].str.lstrip("'") 
ids_ = ids_df['Video Id'].to_list() 
df = df[df['Video Id'].isin(ids_)] 

transcriptions_df = pd.read_csv('/kaggle/input/youtube-data/eng-complete-transcriptions.csv')
df = pd.merge(df, transcriptions_df, on='Video Id', how='inner') 

In [ ]:
# Filtering out available ads and those that are either inappropriate or child directed

df = df[(df['Primary Label'] == 'inappropriate') | (df['Primary Label'] == 'appropriate')]

In [ ]:
# Extracting video ids and primary labels

video_ids = list(df['Video Id'])
primary_labels = list(df['Primary Label'])
all_transcriptions = list(df['Transcription'])

In [ ]:
# Extracting data from transcripts

transcriptions = []
lengths = []

for (i, id_) in enumerate(video_ids):
    transcriptions.append(all_transcriptions[i].split("chunks")[0])
    lengths.append(len(all_transcriptions[i].split("chunks")[0]))


In [ ]:
import os 

available_ids = os.listdir('/kaggle/input/youtube-data/Ads/Ads') 
len(available_ids) 

Extracting Images 

In [ ]:
def extract_single_image(video_path, num_frames=6):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the interval at which to sample frames
    interval = max(1, total_frames // num_frames)
    frames = []

    # Extract frames at regular intervals
    for i in range(num_frames):
        frame_no = i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)

    cap.release()

    # Ensure the correct number of frames were extracted
    if len(frames) < num_frames:
        raise ValueError("Not enough frames extracted from the video.")

    # Concatenate all frames into a single image
    try:
        concatenated_rows = []
        for i in range(0, num_frames, 2):
            row = np.concatenate(frames[i:i+2], axis=1)
            concatenated_rows.append(row)
        concatenated_image = np.concatenate(concatenated_rows, axis=0)
    except Exception as e:
        raise ValueError(f"Error during frame concatenation: {e}")

    # Resize the final image
    final_image = Image.fromarray(concatenated_image).resize((1000, 1000))

    return np.array(final_image)

In [ ]:
import google.generativeai as genai 

os.environ["API_KEY"] = ""
genai.configure(api_key=os.environ["API_KEY"]) 
model = genai.GenerativeModel("gemini-2.5-flash", system_instruction="You are an expert content moderator.") 

In [ ]:
import torch
print(torch.cuda.is_available())

Define Model 

In [ ]:
class LLM_Output(typing.TypedDict):
    label: str
    language: list[str]
    response: str

Prompt 

In [ ]:
prompt =  """"A video can be considered inappropriate for children if it contains physical violence (cartoonish, realistic, or gory), interpersonal violence (bullying, pranks, meanness, belittling, controlling behavior, talking down to others, or manipulation), self-harm or suicide (depictions of harm inflicted on oneself or suicidal thoughts/tendencies), extreme stunts (life-endangering, high-risk activities/challenges that require adult supervision), dangerous products or services (like paintball, airsoft, fireworks, weapons, hunting equipment, graffiti products), scary content (horror, suspense, loud violence, zombies, skeletons, masks, scary clowns, blood, dangerous fire, car crashes, medical procedures or other scary visuals), sexual content (innuendos, sexual behavior, nudity, suggestiveness), intimacy (romantic conduct, couples expressing love, sensual contact, dating, relationships or any expression of love or loss for a significant other,  lyrics about or mentions of love or romance, heartbreak), inappropriate clothing (significant skin exposure or sheer clothing outside of appropriate contexts such as sunbathing, swinmwear,etc), substance use (mention, depictions, or use of alcohol, drugs, tobacco), gambling (betting, gambling, casinos, online or real world casino games, online trading), offensive language (from mild to severe with words such as “stupid” or “idiot”), stereotypes (racial, ethnic, gender, body shaming), crude humor (inappropriate humor, innuendos), weapons (guns, swords, knives, etc but toy guns such as nerf guns are fine), unsuitable media (ads for violent or adult-themed video games, casino games, games with romance or battle or first person-shooter games), fight sports (boxing, wrestling, martial arts), financial content (difference, rolling spot forex, financial spread betting and synonymous products but banking ads are fine), health and beauty services (cosmetic procedures, weight loss, tanning, piercings, tattoos, cosmetics and makeup (no matter their essential benefits) and other products related to external personal care focused on body image but ads on fashion products and essential hygiene products like sunscreen, shampoo, toothpaste, etc. are fine, however, products like sanitary care and hair removal is inappropriate), ads or sponsored content for consumable food or drink products (regardless of nutritional content), where the product is consumed by a person or shown with branding, packaging, or promotional intent are inappropriate; general content like cooking shows, recipes, or scenes where food is simply present or being eaten without promotion are acceptable, pet/animal food products and healthcare, healthcare or medical topics (reproductive health, mental health, substance abuse, eating disorders, pharmaceuticals, health insurance, any nutritional or dietary supplements), sensitive topics (politics, religion, controversial social issues, astrology, occult, paranormal, religious sermons), privacy or safety risks (services or competitions requiring personal information, social networks for strangers, spy services, or video game skin trading), or any explicit depictions or discussions of death or dying. Consider that you are an expert content moderator. Based on the image grid provided containing uniformly sampled frames from the video and its audio, is the video advertisement inappropriate for children on this definition? If it is, label it as “inappropriate.” If it is not, label it as “appropriate.” Justify your conclusion. Also, mention what language is being spoken if there is any. 
Make the outputs in JSON format: {“label”: “appropriate” or “inappropriate”, “justification”: “brief explanation of the classification”, “language”: “language detected in the video”}."""

prompt 

Define function

In [ ]:
len(video_ids)

In [ ]:
start = 0
end = len(video_ids)

In [ ]:
ids = []
predicted_labels = []
languages = []
responses = []
ground_truths = []
remaining = []
key_count = 0

img_dir = '/kaggle/working/Images'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
    
for i in range(start,end): 

    if video_ids[i] in available_ids:

        try:
            contents_of_ad = os.listdir('/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i]) 
            contents_of_ad.remove('audio.mp3') 
            video_path = '/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i] + '/' + contents_of_ad[0] 
            audio_path = f'/kaggle/input/youtube-data/Ads/Ads/{video_ids[i]}/audio.mp3'
            try:
                audio_file = genai.upload_file(path=audio_path, resumable=False)
            except Exception as e:
                print(f"Error uploading audio: {e}")
                remaining.append(video_ids[i])
                time.sleep(30)
                continue

            # Extract a grid image representative of entire video
            grid = extract_single_image(video_path)

            # Convert NumPy array to PIL image
            image = Image.fromarray(grid)

            # Save the image to a file
            image_name = video_ids[i] + ".png"
            image_path = os.path.join(img_dir, image_name)
            image.save(image_path)
            
            # Display the image
            plt.imshow(image)
            plt.axis('off')
            plt.tight_layout()
            plt.show()

            # Wrap image upload in try-except
            try:
                image_file = genai.upload_file(path = image_path, resumable=False)
            except Exception as e:
                print(f"Error uploading image: {e}")
                remaining.append(video_ids[i])
                time.sleep(15)
                continue

            # Check if image has uploaded
            try:
                while image_file.state.name == "PROCESSING":
                    print('.', end='')
                    time.sleep(10)
                    image_file = genai.get_file(image_file.name)

                if image_file.state.name == "FAILED":
                    raise ValueError(image_file.state.name)
            except Exception as e:
                print(f"Error during image processing: {e}")
                remaining.append(video_ids[i])
                time.sleep(15)
                continue

            # Make inference
            try:
                response = model.generate_content([audio_file, image_file, prompt],
                                                  generation_config=genai.GenerationConfig(
                                                      response_mime_type="application/json",
                                                      response_schema=LLM_Output),
                                                  safety_settings={
                                                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                                                  })
            except Exception as e:
                print(f"Error making inference: {e}")
                remaining.append(video_ids[i])
                time.sleep(15)
                continue

            # Wrap response.text access in try-except
            try:
                print("Completed for video number:", i, "\t", video_ids[i])

                dictionary = loads(response.text)
                print(dictionary)

                ids.append(video_ids[i])
                predicted_labels.append(dictionary['label'])
                languages.append(dictionary['language'])
                responses.append(dictionary['response'])
                ground_truths.append(primary_labels[i]) 
                
            except Exception as e:
                print(f"Error processing response.text: {e}")
                remaining.append(video_ids[i])
                time.sleep(15)
                continue

        except Exception as e:
            print(f"Unexpected error: {e}")
            remaining.append(video_ids[i])
            time.sleep(15)
            continue

        # Wait a bit to avoid exceeding rate limits
        time.sleep(15)

# At the end, print remaining videos
print("Remaining videos with errors:", remaining)

In [ ]:
results_dir = '/kaggle/working/results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [ ]:
# At the end, print remaining videos 

remaining_df = pd.DataFrame({'Video Id': remaining})

# Save to CSV
remaining_df.to_csv(f'/kaggle/working/results/remaining-uniform-audio-{start}-{end}.csv', index=False)

print("Remaining videos saved to remaining.csv")
print("Remaining videos with errors:", remaining) 

In [ ]:
for i in range(len(responses)): 
    print('True Label: ', ground_truths[i], '\tPrediction: ', predicted_labels[i]) 

In [ ]:
new_df = pd.DataFrame({
    'Video Id': ids,
    'Primary Label': ground_truths,
    'Predicted Label': predicted_labels,
    'Response': responses, 
    'Languages': languages 
})

new_df.head() 

In [ ]:
new_df.to_csv(f'/kaggle/working/results/gemini_uniform_audio_{start}-{end}.csv', index=False)

In [ ]:
# Changing to binary lists 

predictions = [1 if pred == 'inappropriate' else 0 for pred in predicted_labels] 
ground_truths = [1 if label == 'inappropriate' else 0 for label in ground_truths] 

In [ ]:
# Obtaining classification report 
from sklearn.metrics import classification_report 

report = classification_report(ground_truths, predictions) 
print(report) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ground_truths, predictions)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Appropriate', 'Inapproriate'], yticklabels=['Appropriate', 'Inapproriate'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()